In [1]:
import numpy as np
from collections import Counter, defaultdict, namedtuple
from math import gcd, ceil
import re
import networkx as nx
from dataclasses import dataclass
from matplotlib import pyplot as plt
# plt.imshow(pic)
from functools    import cache

from aocutils.common import to_int, flatten, reverse_dict, zippify, list_multiply, ints
from aocutils.grid import iterate, arr_to_dict, grid_to_dict, neighbors, arr_neighbors, dimensions, manhattan, positive, conv1d, conv2d
from aocutils.maze import bfs, dijkstra, get_path, dfs
from aocutils.math import lcm, gcd, factors, mul_inv
from aocutils.special import find_pattern_in_iter, find_repeat, find_cycle, md5, CFG, ShuntingYard, binarysearch, deduce_matches
from aocutils.visuals import visualize_graph, labelize, cat, animate_grid

In [135]:
lines = open('input.txt').read().splitlines()
lines = [to_int(list(line)) for line in lines]
neigh = arr_neighbors(lines)
maze = grid_to_dict(lines)
conn = {}
A = []
B = []
C = []
D = []
for k,v in maze.items():
    if v != ' ' and v != '#':
        conn[k] = {n for n in neigh[k] if maze[n]!=' ' and maze[n] != '#'}
    if v == 'A':
        A.append(k)
    if v == 'B':
        B.append(k)
    if v == 'C':
        C.append(k)
    if v == 'D':
        D.append(k)
locs = tuple(sorted(A) + sorted(B) + sorted(C) + sorted(D))
state = 0, locs
state

def goal_reached(locs):
    if locs[0] != (2,3) and locs[0] != (3,3):
        return False
    if locs[1] != (2,3) and locs[1] != (3,3):
        return False
    if locs[2] != (2,5) and locs[2] != (3,5):
        return False
    if locs[3] != (2,5) and locs[3] != (3,5):
        return False
    if locs[4] != (2,7) and locs[4] != (3,7):
        return False
    if locs[5] != (2,7) and locs[5] != (3,7):
        return False
    if locs[6] != (2,9) and locs[6] != (3,9):
        return False
    if locs[7] != (2,9) and locs[7] != (3,9):
        return False
    return True


def haspath(o, goal, locs):
    return True

# costs = {0:1, 1:1, 2:10, 3:10, 4:100, 5:100, 6:1000, 7:1000}
costs = {0:1, 1:1, 2:10, 3:10, 4:0, 5:0, 6:0, 7:0}
# costs = {0:1, 1:1, 2:1, 3:1, 4:0, 5:0, 6:0, 7:0}
import heapq
def getnext(state):
    options = []
    cost, locs = state
    # print(locs)
    # occ = [amph for color in locs for amph in color]
    locs = list(locs)
    taken = set(locs)
    for idx, o in enumerate(locs[:4]):
        for n in conn[o]:
            if n not in locs:
                if idx == 0 or idx == 1:
                    if o[0] == 1 and n[0] == 2 and n[1] != 3:
                        print(o, n)
                        continue
                if idx == 2 or idx == 3:
                    if o[0] == 1 and n[0] == 2 and n[1] != 5:
                        continue
                newlocs = locs.copy()
                newlocs[idx] = n
                newcost = costs[idx] + cost
                newlocs = tuple(newlocs)
                options.append(tuple([newcost, newlocs]))
    
    for idx, o in enumerate(locs[4:]):
        newlocs = locs.copy()
        if idx == 0:
            steps = {(1,3), (1,4), (1,5), (1,6), (1,7), (2,7)}
            # steps = {(1,3), (1,4), (1,5), (1,6), (1,7), (2,7), (3,7)}
            if o == (2,3) and not taken & steps:
                newcost = -6* costs[idx+4] + cost
                # assert newcost == -600 + cost
                newlocs[idx+4] = (2,7)
            else:
                continue
        if idx == 1:
            steps = {(1,5), (1,6), (1,7), (2,7), (3,7)}
            # steps = {(1,5), (1,6), (1,7), (2,7)}
            if o == (2,5) and not taken & steps:
                newcost = -5* costs[idx+4] + cost
                # assert newcost == -500 + cost
                newlocs[idx+4] = (3,7)
            else:
                continue
        if idx == 2:
            steps = {(2,3), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8), (1,9), (2,9)}
            # steps = {(2,3), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8), (1,9), (2,9), (3,9)}
            if o == (3,3) and not taken & steps:
                newcost = -9 * costs[idx+4] + cost
                # assert newcost == -9000 + cost
                newlocs[idx+4] = (2,9)
            else:
                continue

        if idx == 3:
            steps = {(2,5), (1,5), (1,6), (1,7), (1,8), (1,9), (2,9), (3,9)}
            # steps = {(2,5), (1,5), (1,6), (1,7), (1,8), (1,9), (2,9)}
            if o == (3,5) and not taken & steps:
                newcost = -8 * costs[idx+4] + cost
                # assert newcost == -8000 + cost
                newlocs[idx+4] = (3,9)
            else:
                continue
        newlocs = tuple(newlocs)
        options.append(tuple([newcost, newlocs]))
    return options
getnext(state)

[(1, ((1, 7), (3, 9), (2, 9), (3, 7), (2, 3), (2, 5), (3, 3), (3, 5))),
 (10, ((2, 7), (3, 9), (1, 9), (3, 7), (2, 3), (2, 5), (3, 3), (3, 5)))]

In [132]:
#export
def dijkstra(connections,start, goal=None):
    """
    Requires a dict with as values a LIST of tuples (neighbor, weight)
    Or a function returning a list of tuples with neighbors and weights per node

    Returns
    if goal == None:    return all paths from start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = [start] # the locations that still need to be visited
    parents = {start[0]: (None,0)}

    def get_path(parents):
        cur = goal
        path = [cur]
        cost = parents[cur][1]
        while cur != start:
            cur = parents[cur][0]
            path.append(cur)
        path.reverse()
        return path,cost
    i = 0
    while frontier:
        i += 1
        if i % 10000 == 0:
            print(len(frontier))
            # pprint(parents)
            # sys.exit()
        # print(parents)
        # print(len(frontier))
        # print('\n\n',frontier,'\n',parents)
        new = heapq.heappop(frontier)
        
        search_cost, search_node = new
        if i % 100 == 0:
            print(new)

        if goal(search_node): 
            print('goal found')
            print(search_cost, search_node)
            return search_cost, search_node, parents
        neighbors = connections(new)
        # print('neigh', len(neighbors), neighbors)
        if neighbors:
            for n in neighbors:
                
                # print(n)
                newcost, newlocs = n


                if newlocs not in seen:
                    # print('n',n)
                    if newlocs not in parents or newcost < parents[newlocs][1]:
                        # print('updating')
                        heapq.heappush(frontier,(newcost, newlocs))
                        # paths[n] = paths[search_node]+[n]
                        parents[newlocs]= search_node,newcost
                            # return paths[goal],parents
        seen.add(search_node)
    
    if not goal: return parents
    elif goal in parents: return get_path(search_node)
    else: return parents
# search_cost, search_node, parents = dijkstra(getnext, state, goal_reached)
# search_cost
cost, _, _ = dijkstra(getnext, state, goal_reached)
print(cost + 18100)


(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(23, ((1, 5), (3, 9), (1, 8), (3, 7), (2, 3), (2, 5), (3, 3), (3, 5)))
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(26, ((1, 9), (2, 9), (1, 8), (3, 7), (2, 3), (2, 5), (3, 3), (3, 5)))
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 7) (2, 7)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 9) (2, 9)
(1, 7) (2, 7)
(1, 

(182,
 ((2, 3), (3, 3), (2, 5), (3, 5), (2, 7), (3, 7), (2, 9), (3, 9)),
 {0: (None, 0),
  ((1, 7),
   (3, 9),
   (2, 9),
   (3, 7),
   (2, 3),
   (2, 5),
   (3, 3),
   (3, 5)): (((2, 7),
    (3, 9),
    (2, 9),
    (3, 7),
    (2, 3),
    (2, 5),
    (3, 3),
    (3, 5)), 1),
  ((2, 7),
   (3, 9),
   (1, 9),
   (3, 7),
   (2, 3),
   (2, 5),
   (3, 3),
   (3, 5)): (((2, 7),
    (3, 9),
    (2, 9),
    (3, 7),
    (2, 3),
    (2, 5),
    (3, 3),
    (3, 5)), 10),
  ((1, 6),
   (3, 9),
   (2, 9),
   (3, 7),
   (2, 3),
   (2, 5),
   (3, 3),
   (3, 5)): (((1, 7),
    (3, 9),
    (2, 9),
    (3, 7),
    (2, 3),
    (2, 5),
    (3, 3),
    (3, 5)), 2),
  ((1, 8),
   (3, 9),
   (2, 9),
   (3, 7),
   (2, 3),
   (2, 5),
   (3, 3),
   (3, 5)): (((1, 7),
    (3, 9),
    (2, 9),
    (3, 7),
    (2, 3),
    (2, 5),
    (3, 3),
    (3, 5)), 2),
  ((1, 7),
   (3, 9),
   (1, 9),
   (3, 7),
   (2, 3),
   (2, 5),
   (3, 3),
   (3, 5)): (((1, 7),
    (3, 9),
    (2, 9),
    (3, 7),
    (2, 3),
    (2, 5),